In [ ]:
# Si no tienes dependencias, ejecuta antes:
# %pip install -q pandas requests plotly kaleido

from datetime import datetime
import math
import json
import requests
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
from pathlib import Path
import streamlit as st

# ==========================
# Parámetros editables
# ==========================
START_YEAR    = 1994
COUNTRY_ISO3  = "MEX"
TITLE_TEXT    = "Crecimiento población"
EXPORT_PNG    = False  # Cambia a True si quieres PNG
OUTPUT_PREFIX = "crecimiento_poblacion"

# Config API World Bank
WB_SERIES = "SP.POP.TOTL"
WB_URL = f"https://api.worldbank.org/v2/country/{COUNTRY_ISO3}/indicator/{WB_SERIES}?format=json&per_page=20000"

# ==========================
# Funciones utilitarias
# ==========================
def fetch_population_worldbank(url: str = WB_URL) -> tuple[pd.DataFrame, dict]:
    """Descarga la serie anual de población total desde la API del World Bank."""
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    data = r.json()
    if not isinstance(data, list) or len(data) < 2 or data[1] is None:
        print("Respuesta de la API:", json.dumps(data, indent=2) if isinstance(data, (list, dict)) else data)
        raise RuntimeError("Respuesta inesperada de la API del World Bank.")

    meta = data[0] if isinstance(data[0], dict) else {}
    lastupdated = meta.get("lastupdated", None)

    rows = []
    for item in data[1]:
        date = item.get("date")
        value = item.get("value")
        if date is None:
            continue
        rows.append({"year": int(date), "value": None if value is None else float(value)})

    df = pd.DataFrame(rows).dropna(subset=["value"]).sort_values("year").reset_index(drop=True)
    return df, {"lastupdated": lastupdated, **meta}

def cagr(first_value: float, last_value: float, years: int) -> float:
    """CAGR (tasa de crecimiento anual compuesta)."""
    if years <= 0 or first_value <= 0:
        return np.nan
    return (last_value / first_value) ** (1.0 / years) - 1.0

def format_es(num: float) -> str:
    """Formatea número entero con separador de miles ','."""
    return f"{int(num):,}"

# =======================================================
# FUNCIÓN CENTRAL DE GRÁFICO (CON ESTILOS INYECTADOS EN STREAMLIT)
# =======================================================
def build_figure(df: pd.DataFrame, start_year: int, title_text: str, lastupdated: str | None, active_palette: list[str], active_font: str) -> tuple[go.Figure, dict]:
    """Construye la figura y devuelve (fig, info) con metadatos."""
    df = df[df["year"] >= start_year].copy()
    if df.empty:
        raise ValueError("No hay datos para el rango solicitado.")

    # --- Obtener colores de la paleta inyectada con fallbacks ---
    BAR_COLOR = active_palette[0]
    EMPHASIS_COLOR = active_palette[1] if len(active_palette) > 1 else "#ff9f18"
    ANNOTATION_COLOR = active_palette[2] if len(active_palette) > 2 else "#889064"
    # -------------------------------------------------------------

    first_year = int(df["year"].min())
    last_year  = int(df["year"].max())
    first_val  = float(df.loc[df["year"] == first_year, "value"].iloc[0])
    last_val   = float(df.loc[df["year"] == last_year, "value"].iloc[0])
    years_gap  = last_year - first_year
    rate       = cagr(first_val, last_val, years_gap)

    # Eje Y: ticks cada 20 millones
    step = 2e7
    ymax = max(step, math.ceil((df["value"].max() + 1) / step) * step)
    tickvals = list(np.arange(0, ymax + 1, step))
    ticktext = [format_es(v) for v in tickvals]

    # Construcción de figura
    pio.templates.default = "plotly_white"
    fig = go.Figure()
    fig.add_bar(
        x=df["year"], y=df["value"],
        marker_color=BAR_COLOR, # <-- Usa el color activo
        marker_line_width=0,
        hovertemplate="Año %{x}<br>Población: %{y:,}<extra></extra>",
        name="Población"
    )

    fig.update_layout(
        title=dict(text=title_text, x=0.5, xanchor="center", y=0.95),
        font=dict(family=active_font, size=14, color="#2e2e2e"), # <-- Usa la fuente activa en Streamlit
        showlegend=False,
        bargap=0.2,
        margin=dict(l=90, r=40, t=70, b=110),
        plot_bgcolor="white",
        paper_bgcolor="white",
    )

    fig.update_xaxes(
        title_text="",
        tickangle=90,
        showgrid=False,
        tickmode="array",
        tickvals=df["year"],
        ticktext=[str(y) for y in df["year"]],
        tickfont=dict(size=12),
        zeroline=False,
    )

    fig.update_yaxes(
        title_text="",
        showgrid=True,
        gridcolor="rgba(0,0,0,0.08)",
        tickmode="array",
        tickvals=tickvals,
        ticktext=ticktext,
        zeroline=False,
    )

    # Texto de la anotación con CAGR
    anot_html = (
        f"Tasa de crecimiento<br>(<b>{first_year} – {last_year}</b>)<br>"
        f"<b><span style='color:{EMPHASIS_COLOR}'>{rate*100:.2f}% anual</span></b>" # <-- Usa color de énfasis
    )

    fig.add_annotation(
        xref="paper", yref="paper", x=0.8, y=1.09,
        xanchor="center", yanchor="middle",
        text=anot_html, showarrow=False, arrowhead=2, arrowsize=1, arrowwidth=1.5,
        arrowcolor=ANNOTATION_COLOR, # <-- Usa color de anotación en Streamlit
        ax=120, ay=40,
        font=dict(size=14, color="#2e2e2e"),
        borderpad=4,
    )

    # ---------- Caption con última fecha ----------
    caption_parts = [f"Último año disponible: {last_year}"]
    if lastupdated:
        try:
            try:
                dt = datetime.strptime(lastupdated[:10], "%Y-%m-%d")
            except Exception:
                dt = datetime.fromisoformat(lastupdated.split("T")[0])
            lastupdated_str = dt.strftime("%Y-%m-%d")
        except Exception:
            lastupdated_str = lastupdated
        caption_parts.append(f"Fuente actualizada: {lastupdated_str}")

    caption_text = " · ".join(caption_parts)

    fig.add_annotation(
        xref="paper", yref="paper",
        x=0.3, y=-0.18,
        xanchor="right", yanchor="top",
        text=f"<span style='font-size:12px;color:#6b6b6b'>{caption_text}</span>",
        showarrow=False,
        align="right"
    )
    # -------------------------------------------

    info = dict(
        first_year=first_year,
        last_year=last_year,
        first_value=first_val,
        last_value=last_val,
        cagr=rate,
        lastupdated=lastupdated
    )
    return fig, info

# ==========================
# Ejecución en Notebook
# ==========================
df, meta = fetch_population_worldbank(WB_URL)

# --- FALLBACKS PARA EJECUCIÓN FUERA DE STREAMLIT ---
DEFAULT_PALETTE = ["#0b132b", "#ff9f18", "#889064"]
DEFAULT_FONT = "Aptos Light, Aptos, Segoe UI, Arial, sans-serif"

# Obtener variables inyectadas en Streamlit (si existen) o usar valores por defecto
PALETTE = globals().get('active_palette', DEFAULT_PALETTE)
FONT = globals().get('active_font', DEFAULT_FONT)

# Llamar a build_figure con las variables de estilo
fig, info = build_figure(
    df,
    start_year=START_YEAR,
    title_text=TITLE_TEXT,
    lastupdated=meta.get("lastupdated"),
    active_palette=PALETTE,
    active_font=FONT
)

# Muestra la figura en el notebook (opcional)
#fig.show()

# Mostrar la gráfica en Streamlit (¡Obligatorio para la app!)
st.plotly_chart(fig, use_container_width=True)

print("\n=== Resumen ===")
print(f"Rango: {info['first_year']}–{info['last_year']} ({info['last_year']-info['first_year']} años)")
print(f"Población inicial: {info['first_value']:,}")
print(f"Población final:   {info['last_value']:,}")

2025-11-29 00:43:52.451 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
2025-11-29 00:43:52.455 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 00:43:52.458 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 00:43:52.459 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 00:43:52.460 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 00:43:52.462 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.



=== Resumen ===
País: MEX
Rango: 1994–2024 (30 años)
Población inicial: 89,259,205.0
Población final:   130,861,007.0
Archivos guardados: crecimiento_poblacion_MEX_20251129.html
